# Descarga de datos y preprocesado
En este código vamos a descargar los datos de las elecciones, rentas y datos demográficos para ser analizados posteriormente.

In [ ]:
!cd .. && pip install .

In [ ]:
import os
import pandas as pd
import numpy as np
import bokeh as bk
import holoviews as hv
from tqdm import tqdm


from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
from bokeh.palettes import brewer

from bokeh.io.doc import curdoc
from bokeh.models import Slider, HoverTool, Select
from bokeh.layouts import widgetbox, row, column


current_dir = os.path.dirname(os.getcwd())
current_dir

## Descarga de datos

Primero descargamos todos los datos. Aquí se incluyen las elecciones de varios años, los datos de renta y datos demográficos.

In [ ]:
from eec import sacar_datos

# Datos de las elecciones
# Estos datos tienen toda la información acerca de las elecciones. Las elecciones incluyen los partidos que se presentan,
# y el número de votos por colegio electoral y partido.
sacar_datos(current_dir + '/datos/congreso_2019_04/', '02201904_MESA.zip', 
            'http://www.infoelectoral.mir.es/infoelectoral/docxl/apliextr/02201904_MESA.zip')

sacar_datos(current_dir + '/datos/congreso_2016_06/', '02201606_MESA.zip', 
            'http://www.infoelectoral.mir.es/infoelectoral/docxl/apliextr/02201606_MESA.zip')

sacar_datos(current_dir + '/datos/congreso_2015_12/', '02201512_MESA.zip', 
            'http://www.infoelectoral.mir.es/infoelectoral/docxl/apliextr/02201512_MESA.zip')

# Datos censales 2016
# Estos datos tienen dos ficheros principales:
# 1) renta_media
# 2) indicadores_demográficos
sacar_datos(current_dir + '/datos/renta_demografía_2016/', '30824.csv',
           "https://www.ine.es/jaxiT3/files/t/es/csv_sc/30824.csv")
sacar_datos(current_dir + '/datos/renta_demografía_2016/', '30832.csv',
           "https://www.ine.es/jaxiT3/files/t/es/csv_sc/30832.csv")
sacar_datos(current_dir + '/datos/renta_demografía_2016/', '30831.csv',
           "https://www.ine.es/jaxiT3/files/t/es/csv_sc/30831.csv")
sacar_datos(current_dir + '/datos/renta_demografía_2016/', '30830.csv',
           "https://www.ine.es/jaxiT3/files/t/es/csv_sc/30830.csv")
sacar_datos(current_dir + '/datos/renta_demografía_2016/', '30828.csv',
           "https://www.ine.es/jaxiT3/files/t/es/csv_sc/30828.csv")
sacar_datos(current_dir + '/datos/renta_demografía_2016/', '30827.csv',
           "https://www.ine.es/jaxiT3/files/t/es/csv_sc/30827.csv")

# Datos censales 2011
# Este fichero contiene muchos datos demográficos; asociados a edad, sexo, inmigración, etc.
sacar_datos(current_dir + '/datos/censos2011_datos/', 'indicadores_seccion_censal_csv.zip',
           "https://www.ine.es/censos2011_datos/indicadores_seccion_censal_csv.zip")
sacar_datos(current_dir + '/datos/censos2011_datos/', 'indicadores_seccen_rejilla.xls',
           "https://www.ine.es/censos2011_datos/indicadores_seccen_rejilla.xls")

# Datos de cartografía de 2011
sacar_datos(current_dir + '/datos/cartografia_2011/', 'cartografia_censo2011_nacional.zip',
           "https://www.ine.es/censos2011_datos/cartografia_censo2011_nacional.zip")

In [ ]:
os.rename(current_dir + '/datos/renta_demografía_2016/30824.csv', 
          current_dir + '/datos/renta_demografía_2016/renta_media.csv')

os.rename(current_dir + '/datos/renta_demografía_2016/30832.csv', 
          current_dir + '/datos/renta_demografía_2016/datos_demograficos_general.csv')

os.rename(current_dir + '/datos/renta_demografía_2016/30831.csv', 
          current_dir + '/datos/renta_demografía_2016/datos_demograficos_relativo_sexo_nacionalidad.csv')

os.rename(current_dir + '/datos/renta_demografía_2016/30830.csv', 
          current_dir + '/datos/renta_demografía_2016/datos_demograficos_relativo_sexo_edad.csv')

os.rename(current_dir + '/datos/renta_demografía_2016/30828.csv', 
          current_dir + '/datos/renta_demografía_2016/datos_demograficos_absoluto_sexo_nacionalidad.csv')

os.rename(current_dir + '/datos/renta_demografía_2016/30827.csv', 
          current_dir + '/datos/renta_demografía_2016/datos_demograficos_absoluto_sexo_edad.csv')

**IMPORTANTE:** para el archivo ``indicadores_seccen_rejilla_.xls`` hay que crear un nuevo documento, ``indicadores.csv``, con los contenidos de la segunda hoja del excels. Los contenidos quedan así.

In [ ]:
indicadores = pd.read_csv(current_dir + '/datos/censos2011_datos/indicadores.csv', encoding='latin1', sep=';', header=None)
indicadores = indicadores.set_index(0)

## Procesado de datos
Ahora vamos a procesar los datos para que queden más legibles y sean más sencillos de manipular.

### Procesado de datos electorales

Para procesar los datos electorales se siguen los siguientes pasos:
- Abrirmos cada archivo ``.DAT`` y asignar nombre de columna a cada elemento de la tabla. Los elementos de las tablas vienen referenciados en el archivo ``FICHEROS.doc``.
- Procesamos los códigos de candidatura. Para procesarlos, los nombres de candidaturas similares se reducen al mismo nombre. Por ejemplo, ``UNIDOS PODEMOS``, ``UNIDAS PODEMOS``, ``ECP-GUANYEM EL CAMBI`` y ``PODEMOS-IU`` se engloban dentro de ``PODEMOS``. Los resultados de candidaturas diferentes se suman. Algunos cambios de candidaturas son tochos, pero también es por agilizar el procesamiento.
- Unificamos los datos de los votos por candidatura, que están en el fichero 10, y los recuentos de votos, votos nulos y votos en blanco, que están en el fichero 9. 
- Reducimos el número de filas quedándonos sólo con la información de las secciones, pues es al final con la que acabaremos trabajando.
- Creamos una columna ``codigo`` que incluya la información de provincia, municipio, distrito y sección para luego cruzarla con otas tablas.

El archivo final es el archivo `resultados_candidaturas.pickle`, que está comprimido. Para ello usaremos el módulo `pandas`.

In [ ]:
from eec import procesar_elecciones

procesar_elecciones(carpeta=current_dir + '/datos/congreso_2019_04/', sufijo='021904')
procesar_elecciones(carpeta=current_dir + '/datos/congreso_2016_06/', sufijo='021606')
procesar_elecciones(carpeta=current_dir + '/datos/congreso_2015_12/', sufijo='021512')

### Procesado de rentas y calificadores demográficos
El formato del archivo es similar al de los datos electorales, indexados por el código de municipio.
Por practicidad, hemos incluido las columnas de los documentos como `pandas.MultiIndex` de modo que se puede hacer una selección de la columna como `df[('A', 'B', 'C')]`.

In [ ]:
from eec import procesar_renta

procesar_renta(carpeta=current_dir + '/datos/renta_demografía_2016/', años=['2016', '2015'])

In [ ]:
from eec import procesar_demografia

procesar_demografia(current_dir + '/datos/renta_demografía_2016/datos_demograficos_general.csv', 
                   ['categoria', 'año'], 11)
procesar_demografia(current_dir + '/datos/renta_demografía_2016/datos_demograficos_relativo_sexo_nacionalidad.csv', 
                   ['sexo', 'nacionalidad', 'renta', 'año'], 8)
procesar_demografia(current_dir + '/datos/renta_demografía_2016/datos_demograficos_relativo_sexo_nacionalidad.csv', 
                   ['sexo', 'nacionalidad', 'renta', 'año'], 8)
procesar_demografia(current_dir + '/datos/renta_demografía_2016/datos_demograficos_relativo_sexo_edad.csv',
                   ['sexo', 'edad', 'renta', 'año'], 8)
procesar_demografia(current_dir + '/datos/renta_demografía_2016/datos_demograficos_absoluto_sexo_nacionalidad.csv',
                   ['sexo', 'nacionalidad', 'renta', 'año'], 8)
procesar_demografia(current_dir + '/datos/renta_demografía_2016/datos_demograficos_absoluto_sexo_edad.csv',
                   ['sexo', 'edad', 'renta', 'año'], 8)

In [ ]:
from eec import procesar_datos_2011

procesar_datos_2011(current_dir + '/datos/censos2011_datos/')

### Procesado de datos geográficos
Para procesar los datos geográficos vamos a necesitar pasar los datos del formato `shapefile` al formato `geojson`. Este último nos va a permitir colorear las secciones y cargarlas en mapas. Para esto, necesitaremos usar la [siguiente web](https://mapshaper.org/), donde subimos el archivo, y lo exportamos como `geojson`. Antes de exportar hacemos dos cosas:
- En la consola escribimos `-proj EPSG:4326`. Esto transforma las coordenadas del mapa a un sistema más internacional y conocido.
- En `simplify` checkeamos `prevent shape removal`, le damos a `apply` y deslizamos el deslizador hasta 50. Cuando salga a la izquierda `X line intersections`, le damos a `repair`.

Guardamos el archivo resultante `SECC_CPV_E_20111101_01_R_INE.json` en la carpeta del `shapefile`.

Como el archivo es bastante grande, para reducir el espacio vamos a reducir las coordenadas de los polígonos, y añadir campos extra.

In [ ]:
# from shapely.geometry import Point, Polygon
import geopandas as gpd

censo_2011 = gpd.read_file(current_dir + '/datos/cartografia_2011/SECC_CPV_E_20111101_01_R_INE.json')

In [ ]:
censo_2011['CUDIS'] = censo_2011['CUMUN'] + censo_2011['CDIS']

Vamos a crear datasets en niveles superiores, y luego vamos a guardar las columnas relevantes para cada dataset.

In [ ]:
distritos_2011 = censo_2011.dissolve('CUDIS', as_index=False)
municipios_2011 = distritos_2011.dissolve('CUMUN', as_index=False)
provincias_2011 = municipios_2011.dissolve('CPRO', as_index=False)
autonomias_2011 = municipios_2011.dissolve('CCA', as_index=False)

In [ ]:
secciones_2011 = censo_2011[['OBJECTID', 'CUSEC', 'NMUN', 'geometry']]
distritos_2011 = distritos_2011[['OBJECTID', 'CUDIS', 'NMUN', 'geometry']]
municipios_2011 = municipios_2011[['OBJECTID', 'CUMUN', 'NMUN', 'geometry']]
provincias_2011 = provincias_2011[['OBJECTID', 'CPRO', 'NPRO', 'geometry']]
autonomias_2011 = autonomias_2011[['OBJECTID', 'CCA', 'NCA', 'geometry']]

In [ ]:
for gs in [secciones_2011, distritos_2011, municipios_2011, provincias_2011, autonomias_2011]:
    gs = gs.reset_index(drop=True)

In [ ]:
secciones_2011.to_file(current_dir + '/datos/cartografia_2011/secciones_2011.json', driver='GeoJSON', encoding='utf-8')
distritos_2011.to_file(current_dir + '/datos/cartografia_2011/distritos_2011.json', driver='GeoJSON', encoding='utf-8')
municipios_2011.to_file(current_dir + '/datos/cartografia_2011/municipios_2011.json', driver='GeoJSON', encoding='utf-8')
provincias_2011.to_file(current_dir + '/datos/cartografia_2011/provincias_2011.json', driver='GeoJSON', encoding='utf-8')
autonomias_2011.to_file(current_dir + '/datos/cartografia_2011/autonomias_2011.json', driver='GeoJSON', encoding='utf-8')

Opcionalmente, y sobre todo para los mapas más grandes, podemos reducir el tamaño de los archivos como lo hemos hecho inicialmente para el archivo de censos.

**NOTA** Hay veces que el `json` generado directamente de ``geopandas`` no va. Si se pasa por ``mapshaper`` vuelve a funcionar.